
필수 노트북:
- [Module 1.0]
    - Option: [Module 1.5]- BERT Input 변환 과정 확인
- [Module 2.0]   
    - Option: [Module 3.0] BERT Pre-trained 모델을 학습 및 평가
    - Option: [Moduel 3.0] 
        - Train을 노트북 셀단위로 실행하여 어떻게 학습이 되는지를 세부적으로 이해할 수 있음
- 3.1.Create-Train_Script.ipynb
    - [Moduel 3.1] Train Script 작성
     


---
Reference
- Chris Fregly, Antje Barth, Book, Data Science on AWS, https://www.oreilly.com/library/view/data-science-on/9781492079385/
    - Source: Data Science on Amazon Web Services
        - https://github.com/data-science-on-aws/workshop
